In [1]:
import os
from langchain_chroma import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import RemoveMessage
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv

_ = load_dotenv(r"D:\pycharm\project\Raglearn\.env")
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.environ["LANGSMITH_API_KEY"]
os.environ['LANGCHAIN_PROJECT'] = 'Order_rag'
os.environ["DASHSCOPE_API_KEY"] = os.environ["QWEN_API_KEY"]

In [2]:
# 实例化Embedding模型
embedding = SentenceTransformerEmbeddings(model_name=r"D:\Model\LLM\cache\bge-large-zh-v1.5")

# api_key = "WqnmFjEFkKaD16qjHtWjWVgzL3HVdyfQ"
# llm = ChatMistralAI(temperature=0,api_key=api_key)
llm_rewrite = ChatTongyi(model="qwen2.5-72b-instruct")
llm = ChatTongyi(model="qwen2.5-72b-instruct")

C:\Users\a\AppData\Local\Temp\ipykernel_4768\938004629.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name=r"D:\Model\LLM\cache\bge-large-zh-v1.5")
C:\Users\a\.conda\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.document_loaders import CSVLoader

def generate_path(folder_path: str) -> list:
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths


def generate_loaders(file_paths: list) -> list:
    loaders = []
    for file_path in file_paths:
        file_type = file_path.split('.')[-1]
        if file_type == 'csv':
            loaders.append(CSVLoader(file_path, encoding='utf-8'))
    return loaders

def exec_load(loaders: list) -> list:
    texts = []
    for loader in loaders:
        texts.extend(loader.load())
    return texts

In [4]:
import sys
from langchain_core.stores import InMemoryByteStore
from langchain.retrievers import MultiVectorRetriever
from langchain_community.vectorstores import FAISS
import faiss
csv_file_path = './data'
file_path = generate_path(csv_file_path)
loaders = generate_loaders(file_path)
txt = exec_load(loaders)
# vectorstore = Chroma(collection_name= "summaries", embedding_function=embedding, persist_directory="./database/chroma")
# index = faiss.IndexFlatL2(len(embedding.embed_query("hello world")))
vectorstore = FAISS.load_local("./database/faiss", embeddings=embedding, allow_dangerous_deserialization = True)

store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
    search_kwargs={"k": 10}
)

with open(r'./data/doc_ids.txt', 'r') as f:
    doc_ids = [line.strip() for line in f]
retriever.docstore.mset(list(zip(doc_ids, txt)))

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool

template = """
根据用户的输入进行判断,如果其意图是希望推荐食物，表意模糊返回relevant值1, 如果出现了明确的菜品名称返回relevant值0; 
如果relevant=0,返回原始输入,如果relevant=1,需要改写根据其点餐意图改写为菜品类目rewrite,其格式为:（初步推荐菜品名称）, 一级类目，二级类目，三级类目(可选);
对于各级类目,可选的属性如下所示:
一级类目包含:['色拉轻食' '主食' '甜点' '菜品' '小吃']
二级类目包含:['健康餐' '其他主食' '其他甜品' '冰沙' '凉皮' '凉菜' '包子' '卤煮/卤味' '寿司' '年糕' '披萨' '汉堡' '汤羹'
 '泡芙' '炸货' '烤货（非炭烤）' '热菜' '粉' '粥' '糕点' '糖水/炖品' '蛋糕' '酥点' '锅贴' '面' '面包' '饭'
 '饭团' '饺子' '饼/囊/馍' '馄饨' '馒头']
三级类目包含：[nan '其他包子' '生煎包' '小笼包' '炸鸡' '其他炸货' '米线' '其他粉' '米粉' '汤面' '其他面' '炒面' '拌面'
 '焖面' '意大利面' '拉面' '乌冬面' '盖浇面' '凉面' '盖浇饭' '焖饭' '其他饭' '泡饭' '拌饭' '白米饭' '煲仔饭'
 '炒饭' '捞饭' '焗饭' '木桶饭' '油饼' '馅饼' '卷饼' '夹饼/夹馍' '手抓饼' '火烧' '春饼']
最后按照以下格式返回，不需要任何其他说明或者回答;
格式,{{'relevant': value(0/1), 'text': input_or_rewrite}}
问题:{input};
格式化回答:
"""

output_parser = StrOutputParser()
prompt = PromptTemplate(template=template, input_variables = ["input"])
summery_chain = prompt|llm_rewrite|output_parser

In [21]:
summery_chain.invoke({'input':"我要点辣的"})

"{'relevant': 1, 'text': '（辣味菜品）, 菜品, 热菜'}"

In [32]:
doc = retriever.invoke("（辣味菜品）, 菜品, 热菜") 
doc


[Document(metadata={'source': './data\\商品.csv', 'row': 922}, page_content='\ufeffitem: 农家特色炒鸡（半只）\ncate_1_name: 菜品\ncate_2_name: 热菜\ncate_3_name: '),
 Document(metadata={'source': './data\\商品.csv', 'row': 923}, page_content='\ufeffitem: 【爆款】无肉不欢随心配\ncate_1_name: 菜品\ncate_2_name: 热菜\ncate_3_name: '),
 Document(metadata={'source': './data\\商品.csv', 'row': 924}, page_content='\ufeffitem: 【稻稻臻选】随心配【25自选5种】\ncate_1_name: 菜品\ncate_2_name: 热菜\ncate_3_name: '),
 Document(metadata={'source': './data\\商品.csv', 'row': 925}, page_content='\ufeffitem: 干锅花菜（微辣）\ncate_1_name: 菜品\ncate_2_name: 热菜\ncate_3_name: '),
 Document(metadata={'source': './data\\商品.csv', 'row': 926}, page_content='\ufeffitem: 新栗子新鲜上市大份（现炒香喷喷不开口）\ncate_1_name: 菜品\ncate_2_name: 热菜\ncate_3_name: '),
 Document(metadata={'source': './data\\商品.csv', 'row': 927}, page_content='\ufeffitem: 葱烤大排\ncate_1_name: 菜品\ncate_2_name: 热菜\ncate_3_name: '),
 Document(metadata={'source': './data\\商品.csv', 'row': 928}, page_content='\ufeffitem: 腊肉豆丝

In [24]:
system_prompt = """
你是一个菜品点单系统,你的任务是根据提供的content条目, 针对老年群体的模糊点餐问题进行帮助点餐,
你的点餐的菜品名称要与content条目中的菜品的item属性完全一致, 且只能在其中选择, 
你的回答不需要进行解释,只需要输出格式化的回答比如,我要点XX等等或者帮我点XX.
content:{content}
question:{input}
answer:
"""
system_tem = PromptTemplate.from_template(system_prompt)

In [25]:
import ast
llm_chain = system_tem | llm | output_parser

def retri_or_return(raw, relevant=0, text=None):
    if relevant == 0:
        return raw

    content = retriever.invoke(text)
    if content:
        content = [i.page_content+'\n' for i in doc]
        content = ''.join(content)
        return llm_chain.invoke({'content':content, 'input':raw})

def parse_and_fun(input_str):
    parsed_data = ast.literal_eval(input_str)
    return retri_or_return(raw, **parsed_data)

In [26]:
all_chain = summery_chain | parse_and_fun

In [27]:
raw = "我要点辣的"
all_chain.invoke({'input':raw})

'我要点干锅花菜（微辣）'